Scrape from Reddit:
Depression
Suicide Ideation
Anxiety
PTSD
Eating Disorder
Body Dysmorphia
plus a Non-mental health condition indicative class

In [1]:
!pip install psaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import csv
import time
import pandas as pd
import requests as req
import os
import sys
import numpy as np
import datetime
import json
from psaw import PushshiftAPI


Anxiety --> r/Anxiety
Depression --> r/Depression
PTSD --> r/PTSD
Eating Disorder --> r/EDAnonymous
Body Dysmorphia --> r/BodyDysmorphia
Suicide --> r/SuicideWatch

In [3]:
api = PushshiftAPI()


In [4]:
subreddits = ['Diary',
              'deardiary',
              'DiaryOfARedditor'
              ]

In [5]:
filtered = ['title','selftext', 'subreddit']

In [6]:
# method to fetch posts from specific subreddits, scrape 200k most recent posts
def data_scraper(subreddit, limit_size, filters):
  posts = list(api.search_submissions(subreddit=subreddit,limit=limit_size, filter=filters))
  df = pd.DataFrame([thing.d_ for thing in posts])

  return df

In [7]:
full_df = pd.DataFrame()

In [8]:
for sub in subreddits:
  sub_df = data_scraper(sub,100000,filtered)
  full_df = pd.concat([full_df,sub_df])
  print(sub)
  print(sub_df.shape)

/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:252: UserWarning: Not all PushShift shards are active. Query results may be incomplete
  warnings.warn(shards_down_message)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:192: UserWarning: Got non 200 code 429
  warnings.warn("Got non 200 code %s" % response.status_code)
/usr/local/lib/python3.7/dist-packages/psaw/PushshiftAPI.py:180: UserWarning: Unable to connect to pushshift.io. Retrying after backoff.
  warnings.warn("Unable to connect to pushshift.io. Retrying after backoff.")


Diary
(7935, 5)
deardiary
(411, 5)
DiaryOfARedditor
(4556, 5)


In [9]:
full_df.head()

,created_utc,selftext,subreddit,title,created
0,1654248812,I feel more and more weak day by day. I feel a...,Diary,Gut punch after gut punch.,1.654249e+09
1,1654240185,I really want to do more. Push myself until I ...,Diary,I want to do more,1.654240e+09
2,1654222265,Do I like him? Sometimes I think it’s a simple...,Diary,Romance vent/ self talk,1.654222e+09
3,1654216553,Shitty bloody day today. \n\nI was still reeli...,Diary,02/06/2022,1.654217e+09
4,1654187279,To whoever cosmic being that decided to give m...,Diary,To you,1.654187e+09


In [10]:
full_df = full_df.drop(columns=['created_utc','created'])
full_df = full_df[['title','selftext', 'subreddit']]
full_df.head()

,title,selftext,subreddit
0,Gut punch after gut punch.,I feel more and more weak day by day. I feel a...,Diary
1,I want to do more,I really want to do more. Push myself until I ...,Diary
2,Romance vent/ self talk,Do I like him? Sometimes I think it’s a simple...,Diary
3,02/06/2022,Shitty bloody day today. \n\nI was still reeli...,Diary
4,To you,To whoever cosmic being that decided to give m...,Diary


In [11]:
full_df.shape

(12902, 3)

In [12]:
df = full_df
df.dropna(inplace=True)
df.shape

(12827, 3)

In [13]:
# remove any posts that have [removed] or [deleted] in selftext
indx = df.index[df.selftext == '[removed]'].tolist()
print(indx)
df.drop(indx, inplace=True)
df.shape

[13, 60, 265, 266, 267, 268, 277, 302, 303, 306, 315, 342, 417, 456, 457, 886, 1047, 1063, 1110, 1266, 1378, 1389, 1418, 1524, 1612, 1654, 1736, 1737, 1755, 1827, 1830, 1935, 1942, 2038, 2174, 2183, 2184, 2212, 2769, 2771, 2848, 2937, 2939, 2941, 3012, 3050, 3094, 3104, 3665, 4740, 4743, 4831, 4858, 5040, 5436, 5841, 6429, 7632, 111, 3, 9, 10, 11, 14, 15, 19, 20, 21, 22, 25, 34, 35, 40, 46, 48, 53, 54, 55, 56, 57, 60, 61, 63, 67, 73, 74, 75, 76, 78, 80, 81, 86, 87, 95, 97, 103, 106, 108, 110, 111, 115, 119, 121, 123, 124, 130, 131, 137, 138, 142, 143, 150, 153, 162, 166, 171, 182, 184, 185, 189, 199, 208, 214, 215, 219, 226, 229, 234, 238, 242, 250, 258, 261, 265, 267, 268, 285, 287, 294, 301, 303, 323, 325, 326, 328, 334, 339, 340, 341, 346, 350, 351, 353, 354, 358, 359, 360, 363, 377, 378, 379, 380, 384, 387, 388, 389, 392, 393, 394, 398, 399, 402, 403, 404, 407, 411, 425, 426, 431, 434, 437, 438, 440, 444, 449, 450, 451, 457, 459, 460, 461, 462, 466, 471, 473, 476, 477, 480, 481, 48

(10028, 3)

In [14]:
indx = df.index[df.selftext == '[deleted]'].tolist()
print(indx)
df.drop(indx, inplace=True)
df.shape

[955, 1385, 1971, 2496, 2499, 2615, 2928, 2932, 3132, 3190, 3251, 3254, 3324, 3367, 3375, 3577, 3578, 3599, 4101, 4104, 4588, 5414, 5648, 5653, 5708, 5721, 5745, 5767, 5775, 5776, 5777, 5779, 5780, 5781, 5782, 5785, 5790, 5791, 5794, 5820, 5821, 5823, 5839, 5842, 5872, 5886, 5888, 5894, 5896, 5897, 5898, 5901, 5905, 5932, 5939, 5951, 5954, 5957, 5958, 5978, 5980, 5987, 5991, 5999, 6001, 6003, 6017, 6027, 6055, 6061, 6064, 6066, 6087, 6099, 6102, 6144, 6149, 6180, 6191, 6210, 6221, 6320, 6343, 6356, 6358, 6494, 6513, 6591, 6666, 6687, 6688, 6756, 6800, 6831, 6852, 6872, 6879, 6890, 6926, 6929, 6932, 6935, 6938, 6941, 6944, 6946, 6949, 6967, 6999, 7001, 7003, 7004, 7048, 7051, 7071, 7072, 7073, 7082, 7085, 7088, 7106, 7113, 7135, 7136, 7151, 7172, 7184, 7268, 7270, 7304, 7312, 7314, 7321, 7325, 7327, 7333, 7354, 7359, 7370, 7373, 7380, 7381, 7390, 7393, 7394, 7399, 7400, 7402, 7406, 7409, 7413, 7416, 7419, 7423, 7424, 7426, 7429, 7430, 7431, 7432, 7433, 7435, 7436, 7439, 7440, 7442, 7445

(9392, 3)

In [15]:
# drop duplicates
df.drop_duplicates(keep='first',inplace=True)
df.shape

(9357, 3)

Now we assign a class label to match each subreddit scraped


In [ ]:
df['subreddit'].unique()

In [ ]:
sub_conditions = [
                  (df['subreddit'] == 'SuicideWatch'),
                  (df['subreddit'] == 'depression'),
                  (df['subreddit'] == 'ptsd'),
                  (df['subreddit'] == 'Anxiety'),
                  (df['subreddit'] == 'EDAnonymous'),
                  (df['subreddit'] == 'BodyDysmorphia')
]

In [ ]:
classes_names = [
                 'Suicide',
                 'Depression',
                 'PTSD',
                 'Anxiety',
                 'Eating Disorder',
                 'Body Dysmorphia'
]

In [ ]:
df['class'] = np.select(sub_conditions, classes_names)
df.head()

In [ ]:
df.drop(columns=['subreddit'], inplace=True)
df.head()

,title,selftext,class
0,I should have killed myself instead of moving out,I'm so fucking lame. I have re-writed this pos...,Suicide
3,Killing myself next chance I get,"Screw my plans, fuck methods, I'm leaping off ...",Suicide
5,What has happened has happened,"First off i am not in the right mood to joke, ...",Suicide
6,The only thing that stopped me before was thin...,,Suicide
10,I’m stupid,"I’m stupid, I’m worthless, and my mistakes jus...",Suicide


In [16]:
df.reset_index()
df.head()
df.tail()
df.shape

(9357, 3)

In [18]:
from google.colab import files
df.to_csv('dataset_journal_entries.csv',encoding='utf-8')
files.download("dataset_journal_entries.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>